# Demo workflow

Demonstrate simple workflow for case where only pose data is available (no behavior labels)

In [ ]:
from ethome import create_dataset, interpolate_lowconf_points
from ethome.io import get_sample_nwb_paths
from ethome.unsupervised import compute_umap_embedding
from ethome.plot import plot_embedding, interactive_tracks

Gather some sample tracking files to play with

In [ ]:
nwb_file = get_sample_nwb_paths()

Nwb files already contain metedata about the tracking, so we don't have to provide this ourselves:

In [ ]:
#%% Create dataset
dataset = create_dataset(nwb_file)
dataset

In [ ]:
dataset.pose.body_parts

Checkout the tracks with a widget

In [ ]:
%matplotlib inline
filename = dataset.metadata.videos[0]
interactive_tracks(dataset,
                   filename)

Smooth over low-confidence points

In [ ]:
interpolate_lowconf_points(dataset)

Now create features on this dataset. Can use pre-built featuresets, or make your own. 
As we don't have a resident-intruder setup, here we use generic featuresets. 

In [ ]:
dataset.features.add('distances')

## Unsupervised learning

In [ ]:
#%%################
## Dim reduction ##
###################

embedding = compute_umap_embedding(dataset, dataset.features.active, N_rows = 10000)
dataset[['embedding_0', 'embedding_1']] = embedding

In [ ]:
fig, ax = plot_embedding(dataset) 

## Post processing

Now we have our model we can make a video of its predictions. Provide the column names whose state we're going to overlay on the video, along with the directory to output the videos.

NOTE: need to have provided 'video' column in the metadata to make movies.

In [ ]:
dataset.io.save_movie(['label', 'prediction'], '.')